In [3]:
from typing import Any, Tuple
import yaml
import numpy as np
import itertools

In [4]:
def sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    values = loader.construct_sequence(node.value[0][1])
    return {"type": "sweep", "values": values}


def coupled_sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    target = node.value[0][1].value
    values = loader.construct_sequence(node.value[1][1])
    return {"target": target, "values": values, "type": "coupled-sweep"}


def get_loader():
    loader = yaml.SafeLoader
    loader.add_constructor("!sweep", sweep_constructor)
    loader.add_constructor("!coupled-sweep", coupled_sweep_constructor)
    return loader

In [5]:
cfg = yaml.load(open("./config.yaml"), Loader=get_loader())

In [6]:
cfg

{'sweepname': {'type': 'sweep', 'values': [1, 2, 3, 4, 5]},
 'othersweep': {'type': 'sweep', 'values': ['x', 'y', 'z']},
 'coupled_sweep': {'target': 'sweepname',
  'values': [6, 7, 8, 9, 10],
  'type': 'coupled-sweep'},
 'nested': {'a': {'b': {'type': 'sweep', 'values': [11, 12, 13, 14, 15]},
   'c': 'x'}},
 'd': {'target': 'othersweep',
  'values': ['a', 'b', 'c'],
  'type': 'coupled-sweep'}}

In [ ]:
class Run:
    def __init__(self, config: dict):
        self.config = config
        self.run_configs = []
        sweep_targets = {}
        coupled_targets = {}
        self._cfg_helper(self.config, sweep_targets, coupled_targets)

        print(sweep_targets)
        print(coupled_targets)

        self.run_configs = self._construct_run_configs(sweep_targets, coupled_targets)
        print(self.run_configs)

    def _cfg_helper(
        self, cfg_node: dict, sweep_targets, coupled_targets
    ) -> Tuple[list[Any], list[Any]]:
        for key, node in cfg_node.items():
            if isinstance(node, dict) and "type" in node:
                if node["type"] == "sweep":
                    sweep_targets[key] = {"key": key, "values": node["values"]}
                elif node["type"] == "coupled-sweep":
                    coupled_targets[key] = {
                        "target": node["target"],
                        "values": node["values"],
                    }
            elif isinstance(node, dict):
                self._cfg_helper(node, sweep_targets, coupled_targets)
            else:
                continue  # we ignore leafs

        return sweep_targets, coupled_targets

    def _cartesian_helper(self, key_current, current_set, key_next, next_set):
        for v in current_set:
            for w in next_set:
                pass


    def _construct_run_configs(self, sweep_targets, coupled_targets):
        # make list of tuple values
        elements = []
        value = []
        i = 0
        j = 0
        lists = [v["values"] for k, v in sweep_targets.items()]

        return elements

In [46]:
runner = Run(cfg)

{'sweepname': {'key': 'sweepname', 'values': [1, 2, 3, 4, 5]}, 'othersweep': {'key': 'othersweep', 'values': ['x', 'y', 'z']}, 'b': {'key': 'b', 'values': [11, 12, 13, 14, 15]}}
{'coupled_sweep': {'target': 'sweepname', 'values': [6, 7, 8, 9, 10]}, 'd': {'target': 'othersweep', 'values': ['a', 'b', 'c']}}
11
12
13
14
15
[]


In [33]:
v = [1,2,3,4]
w = ["a", "b", "c"]

for v_ in v:
    for w_ in w:
        print(v_, w_)

1 a
1 b
1 c
2 a
2 b
2 c
3 a
3 b
3 c
4 a
4 b
4 c
